In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True


fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-2b-bnb-4bit",
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,

)

==((====))==  Unsloth: Fast Mistral patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep


In [ ]:
EOS_TOKEN = tokenizer.eos_token
import pandas as pd
from datasets import Dataset

# Load the CSV file
df = pd.read_csv('/content/mega_dataset3.csv', encoding='utf-8')
#if the row has no 'topic' feature then make it an empty string
df['topic'] = df['topic'].fillna('')

# concat the features
df['text'] = "Topic: [" + df['topic'] + "] Subreddit: [" + df['Subreddit'] + "] Title: ["+ df['title']+ "] Body: [" + df['body'] + "]"
# Convert the 'text' column to string type
df['text'] = df['text'].astype(str)
# Create the dataset from the combined text
dataset = Dataset.from_dict({'text': df['text'].tolist()})

def formatting_func(example):
    return example["text"] + EOS_TOKEN


<a name="Train"></a>
### Train the model



In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    tokenizer = tokenizer,
    max_seq_length = max_seq_length,
    packing = True,
    formatting_func = formatting_func,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.05,
        max_grad_norm = 1.0,
        num_train_epochs = 1,
        gradient_checkpointing = True,
        learning_rate = 2e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.1,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Generating train split: 0 examples [00:00, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:342: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,109 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.108800
2,2.100200
3,2.258700
4,2.188500
5,2.059400
6,2.190600
7,2.158200
8,2.224000
9,2.238000
10,1.997400


<a name="EXPERIMENTATION"></a>
### **EXPERIMENTATION**



In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [botswana] Subreddit: [bipolar]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [botswana] Subreddit:[bipolar] Title: [I’m a 22 year old male with bipolar disorder and 
I’m looking for a friend] Body: [I’m a 22 year old male with bipolar disorder and I’m looking for a 
friend. I’m not looking for a romantic relationship, just a friend. I’m looking for someone who is 
understanding and supportive. I’m looking for someone who is willing to listen and be there for me. I’m looking 
for someone who is willing to help me through my tough times. I’m looking for someone who is willing 
to be there for me when I need them. I’m looking for someone who is willing to be there for me when 
I’m feeling down. I’m looking for someone who is willing to be there for me when I’m feeling lonely. 
I’m looking for someone who is willing to be there for me when I’m feeling lost. I’m looking for 
someone who is willing to be there for me when I’m feeling hopeless. I’m looking for someone who is 
willing to be there for me when I’m feeling like I’m not good enough. I’m looking 

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [paranoid] Subreddit: [bipolar]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [paranoid] Subreddit:[bipolar]

I'm a 23 year old male, and I've been diagnosed with 
bipolar disorder. I've been on medication for about 2 years now, and I've been doing pretty well. I've 
been in a relationship with my girlfriend for about 2 years now, and we've been living together for 
about a year.

I've been feeling really paranoid lately, and I'm not sure why. I've been feeling like my 
girlfriend is cheating on me, and I'm not sure why. I've been feeling like she's been lying to me, and I'm 
not sure why. I've been feeling like she's been hiding something from me, and I'm not sure why.

I've 
been feeling like she's been acting differently around me, and I'm not sure why. I've been feeling like 
she's been avoiding me, and I'm not sure why. I've been feeling like she's been ignoring me, and I'm not 
sure why.

I've been feeling like she's been acting differently around me, and I'm not sure why.

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [lebron] Subreddit: [bipolar]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [lebron] Subreddit:[bipolar] Title: [I’m a 20 year old male and I’m bipolar. I’m also a 
huge basketball fan and I’m a die hard lebron fan. I’m also a huge lebron fan. I’m also a huge lebron 
fan. I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also a 
huge lebron fan. I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also a huge lebron fan. 
I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also a huge 
lebron fan. I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also 
a huge lebron fan. I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also a huge

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [girlfriend] Subreddit: [bipolar]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [girlfriend] Subreddit:[bipolar] Title: help me
Body:



male in to help me with my 
girlfriend. She has bipolar disorderalar disorder and I'm not sure how to


'm not sure how to help her. 
She'sipolar and I'm not sure how to help her. She's been having a hard time lately and I' not sure how to help 
her. She's been having a hard time lately and I'm not sure how to help her. She's been having a hard 
time lately and I'm not sure how to help her. She's been having a hard time lately and I'm not sure how 
to help her. She's been having a hard time lately and I'm not sure how to help her. She's been having 
a hard time lately and I'm not sure how to help her. She's been having a hard time lately and I'm 
not sure how to help her. She's been having a hard time lately and I'm not sure how to help her. She's 
been having a hard time lately and I'm not sure how to help her. She's been having a hard time lately 
and I

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [botswana] Subreddit: [bipolar]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [botswana] Subreddit:[bipolar] Title: [I'm a 20 year old male with bipolar disorder. 
I'm currently in a manic episode and I'm not sure what to do. I'm scared.] Body: [I'm a 20 year old 
male with bipolar disorder. I'm currently in a manic episode and I'm not sure what to do. I'm scared.


I've been diagnosed with bipolar disorder for about 2 years now. I've been on medication for about a 
year and a half. I've been doing pretty well for the most part. I've had a few episodes of depression 
and a few episodes of mania, but nothing too bad.

But now I'm in a manic episode and I'm not sure 
what to do. I'm scared.

I'm not sure if I should go to the hospital or not. I'm not sure if I should 
tell my parents or not. I'm not sure if I should tell my friends or not. I'm not sure if I should tell 
my girlfriend or not. I'm not sure if I should tell anyone or not.

I'

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [crossy road] Subreddit: [bipolar]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [crossy road] Subreddit:[bipolar] Title: [I’m so tired of being bipolar] Body: [I’m so 
tired of being bipolar. I’m tired of the stigma, the discrimination, the lack of understanding, the lack 
of support, the lack of resources, the lack of treatment options, the lack of research, the lack of 
funding, the lack of awareness, the lack of education, the lack of compassion, the lack of empathy, the 
lack of understanding, the lack of support, the lack of resources, the lack of treatment options, the 
lack of research, the lack of funding, the lack of awareness, the lack of education, the lack of 
compassion, the lack of empathy, the lack of understanding, the lack of support, the lack of resources, the 
lack of treatment options, the lack of research, the lack of funding, the lack of awareness, the lack 
of education, the lack of compassion, the lack of empathy, the lack of understanding, the lack of 
support, the lack of resources, the lack of treatment options, the lack of re

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [chatgpt] Subreddit: [bipolar]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [chatgpt] Subreddit:[bipolar] Title me lot, but I’m still scared.

I’m a 20 year old 
male, and I’ve been diagnosed with bipolar disorder for about 2 years now. I’ve been on medication for 
about 2  my whole life, and I’ve been on medication off for about 2 years now. I’m still scared of the 
medication, but because I’m still scared of the medication, I’m still scared of the medication. I’m still 
scared of the medication, but I’m still scared of the medication. I’m still scared of the medication, but 
I’m still scared of the medication. I’m still scared of the medication, but I’m still scared of the 
medication. I’m still scared of the medication, but I’m still scared of the medication. I’m still scared of 
the medication, but I’m still scared of the medication. I’m still scared of the medication, but I’m 
still scared of the medication. I’m still scared of the medication, but I’m still scared of the 
medication. I’m still scared of the medication, but I’m still scared of t

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [botswana] Subreddit: [cmv]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [botswana] Subreddit:[cmv] Title: [CMV] I think that the Botswana government is right 
to ban the sale of alcohol to foreigners. Body: [cmv] I think 

KeyboardInterrupt: 

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [crossy road] Subreddit: [cmv]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [crossy road] Subreddit:[cmv] Title: [CMV] I think the game Crossy Road is a ripoff of 
Flappy Bird. Body: [cmv] I think the game Crossy Road is a ripoff of Flappy Bird. I think the game Crossy 
Road is a ripoff of Flappy Bird. I think the game Crossy Road is a ripoff of Flappy Bird. I think the 
game Crossy Road is a ripoff of Flappy Bird. I think the game Crossy Road is a ripoff of Flappy Bird. I 
think the game Crossy Road is a ripoff of Flappy Bird. I think the game Crossy Road is a ripoff of 
Flappy Bird. I think the game Crossy Road is a ripoff of Flappy Bird. I think 

KeyboardInterrupt: 

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [israel] Subreddit: [bipolar]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [israel] Subreddit:[bipolar] Title: [I’m a 22 year old Israeli man and I’m bipolar. 
AMA] Body: [I’m a 22 year old Israeli man and I’m bipolar. I’ve been diagnosed with bipolar disorder 
for about 3 years now. I’ve been on and off medication for about 2 years now. I’ve been in and out of 
therapy for about 2 years now. I’ve been in and out of the hospital for about 2 years now. I’ve been in 
and out of the psychiatric ward for about 2 years now. I’ve been in and out of the psych ward for 
about 2 years now. I’ve been in and out of the psych ward for about 2 years now. I’ve been in and out of 
the psych ward for about 2 years now. I’ve been in and out of the psych ward for about 2 years now. 
I’ve been in and out of the psych ward for about 2 years now. I’ve been in and out of the psych ward 
for about 2 years now. I’ve been in and out of the psych ward for about 2 years now

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Given a Topic and a Subreddit, generate a Title and a body", # instruction
        "Topic: [botswana] Subreddit: [askwomenover30]", # input
        "Title: [", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Given a Topic and a Subreddit, generate a Title and a body

### Input:
Topic: [botswana] Subreddit: [askwomenover30]

### Response:
Title: [Botswana: A Country of Opportunity for Women Over 30]

Body: [Botswana is a country that offers many opportunities for women over 30. With a growing economy and a focus on gender equality, Botswana is an ideal place for women to pursue their career goals and personal aspirations. The country has a strong education system, with many universities and colleges offering a wide range of courses and programs. Women over 30 can also find plenty of job opportunities in Botswana, with many companies and organizations actively seeking qualified candidates. In addition, the country has a vibrant culture


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [Pasta] Subreddit: [AITAH]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [Pasta] Subreddit:[AITAH] Title'murica' Body: [deleted]



---
















<unk>





























































































































KeyboardInterrupt: 

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [Pasta] Subreddit: [AITAH]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Exception in thread Thread-11 (_fast_generate):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/unsloth/models/llama.py", line 976, in _fast_generate
    return generate(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/peft/peft_model.py", line 1190, in generate
    outputs = self.base_model.generate(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py", line

<s> Topic: [Pasta] Subreddit:[AITAH] Title: [AITAH for not wanting to go to my friend's wedding?] 
Body: [I (25F) have been friends with my friend (25F) for 10 years. We met in college and have been 
close ever since. She is getting married in a few months and has invited me to the wedding. I 

KeyboardInterrupt: 

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [Botswana] Subreddit: [Askwomenover30]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [Botswana] Subreddit:[Askwomenover30] Title: [I’m 30 and I’m not sure if I’m ready to 
be a mother] Body: [I’m 30 and I’m not sure if I’m ready to be a mother. I’m not sure if I want to be 
a mother. I’m not sure if I’m ready to be a mother. I’m not sure if I’m ready to be a mother. I’m 
not sure if I’m ready to be a mother. I’m not sure if I’m ready to be a mother. I’m not sure if I’m 
ready to be a mother. I’m not sure if 

KeyboardInterrupt: 

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [botswana] Subreddit: [jokes]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [botswana] Subreddit:[jokes] Title: [Botswana] Body: [I'm a bot, and this is a 
test.]</s>

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [Botswana] Subreddit: [Jokes]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [Botswana] Subreddit:[Jokes] Title: [A man walks into a bar] Body: [A man walks into a 
bar and orders a drink. The bartender asks him, "What's your name?" The man replies, "I'm called 'The 
Man Who Walks Into Bars.'" The bartender is confused and asks, "Why do you have that name?" The man 
says, "It's a long story. I'll tell you if you buy me a drink." The bartender agrees and the man tells 
his story. "I was born in a small village in Botswana. My father was a farmer and my mother was a 
housewife. I had a happy childhood, but I always felt like I was different from the other kids. I didn't fit 
in. When I was 18, I decided to leave my village and travel the world. I visited many countries and 
met many people, but I never felt like I belonged anywhere. Then one day, I was walking through a city 
and I saw a sign that said 'Bar.' I walked in and ordered a drink. The bartender asked me my name and 
I said, 'I'm called 'The Man Who Walks Into Bars.'" The

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [Botswana] Subreddit: [bipolar]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [Botswana] Subreddit:[bipolar]

I'm a 23 year old male from Botswana. I've been 
diagnosed with bipolar disorder and I'm currently on medication. I'm not sure if I'm in a manic or 
depressive episode but I'm feeling very down and I'm not sure what to do. I'm not sure if I'm in a manic or 
depressive episode but I'm feeling very down and I'm not sure what to do. I'm not sure if I'm in a manic or 
depressive episode but I'm feeling very down and I'm not sure what to do. I'm not sure if I'm in a manic or 
depressive episode but I'm feeling very down and I'm not sure what to do. I'm not sure if I'm in a manic or 
depressive episode but I'm feeling very down and I'm not sure what to do. I'm not sure if I'm in a manic or 
depressive episode but I'm feeling very down and I'm not sure what to do. I'm not sure if I'm in a manic or 
depressive episode

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [lebron james] Subreddit: [tifu]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [lebron james] Subreddit:[tifu] Title: [AITAH for not wanting to go to my friend’s 
wedding?] Body:

I (25M) have been friends with my friend (25M) for 10 years. We met in high school and have 
been close friends ever since. We have been through a lot together and have always been there for each 
other.

Recently, my friend got engaged and is planning his wedding. He has asked me to be his best man 
and I have agreed. However, I am not excited about 

KeyboardInterrupt: 

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [christ] Subreddit: [atheism]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [christ] Subreddit:[atheism] Title: 
[Atheism]<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><un

In [1]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [lebron james] Subreddit: [QAnonCasualties]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Exception ignored in: <function _xla_gc_callback at 0x7ce54c628dc0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


NameError: name 'tokenizer' is not defined

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [lebron] Subreddit: [bipolar]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [lebron] Subreddit:[bipolar] Title: [I’m a 20 year old male and I’m bipolar. I’m also a 
huge basketball fan and I’m a die hard lebron fan. I’m also a huge lebron fan. I’m also a huge lebron 
fan. I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also a 
huge lebron fan. I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also a huge lebron fan. 
I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also a huge 
lebron fan. I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also 
a huge lebron fan. I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also a huge

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [lebron james] Subreddit: [tifu]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [lebron james] Subreddit:[tifu] Title [AITAH] I (25M) told my girlfriend (24F) that I 
don in college and she got mad

Body:.

I (25M) told my girlfriend (24F) that I don't like LeBron 
James (40M) and she got mad.

I'm a big fan of the NBA and I've been following the league for a James. 
I0he I.

♡

I've always been a fan of the Cleveland Cavaliers and I've been following the team since I 
was a kid. I've always been a fan of LeBron James and I've always been a fan of the team.

I've always 
been a fan of the team and I've always been a fan of the team. I've always been a fan of the team and 
I've always been a fan of the team. I've always been a fan of the team and I've always been a fan of 
the team. I've always been a fan of the team and I've always been a fan of the team. I've always been 
a fan of the team and I

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Given a Topic and a Subreddit, generate a Title and a body", # instruction
        "Topic: [chatgpt] Subreddit: [bipolar]", # input
        "Title: [", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGiven a Topic and a Subreddit, generate a Title and a body\n\n### Input:\nTopic: [chatgpt] Subreddit: [bipolar]\n\n### Response:\nTitle: [ChatGPT and Bipolar Disorder: A Comprehensive Guide]\n\nBody:\n\nBipolar disorder is a mental health condition that affects millions of people around the world. It is characterized by extreme mood swings, ranging from manic episodes of high energy and euphoria to']

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [Lebron James] Subreddit: [aitah]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [Lebron James] Subreddit:[aitah] Title: [AITAH for not wanting to go to my friend's 
wedding?] Body: [I (25M) have been friends with my friend (25M) for 10 years. We met in high school and have 
been best friends ever since. We have been through a lot together and have always been there for each 
other.

Recently, my friend got engaged and is planning his wedding. He has asked me to be his best man 
and I have agreed. However, I am not excited about the wedding and I am not sure if I want to go.

I 
have always been a bit of a homebody and I am not really into big parties or events. I am also not 
really into weddings and I am not sure if I want to spend my time and money on something that I am not 
really interested in.

I have told my friend that I am not really excited about the wedding and I am not 
sure if I want to go. He has been understanding and has said that I don't have to go if I don't want 
to.

I am not sure if I am the asshole for not wanting to go to my frien

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [botswana] Subreddit: [ama]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [botswana] Subreddit:[ama] Title: [AMA Request: Botswana] Body: [I'm a 20 year old 
American who is currently living in Botswana. I've been here for 6 months and I'm loving it. I'm a student 
at the University of Botswana and I'm majoring in International Relations. I'm also a member of the 
Botswana National Youth Council. I'm interested in learning more about the country and its people. I'm also 
interested in learning more about the culture and the history of the country. I'm also interested in learning 
more about the economy and the politics of the country. I'm also interested in learning more about the 
education system of the country. I'm also interested in learning more about the healthcare system of the 
country. I'm also interested in learning more about the transportation system of the country. I'm also 
interested in learning more about the tourism industry of the country. I'm also interested in learning more 
about the agriculture industry of the country. I'm al

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [lebron] Subreddit: [tifu]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [lebron] Subreddit:[tifu] Title: [Tifu by not being a lebron fan] Body: [I'm a huge 
basketball fan and I've been following the NBA for a long time. I've always been a fan of the Cleveland 
Cavaliers and I've always been a fan of LeBron James. I've always been a fan of the Cavs and I've always 
been a fan of LeBron James. I've always been a fan of the Cavs and I've always been a fan of LeBron 
James. I've always been a fan of the Cavs and I've always been a fan of LeBron James. I've always been a 
fan of the Cavs and I've always been a fan of LeBron James. I've always been a fan of the Cavs and 
I've always been a fan of LeBron James. I've always been a fan of the Cavs and I've always been a fan 
of LeBron James. I've always been a fan of the Cavs and I've always been a fan of LeBron James. I've 
always been a fan of the Cavs and I've always

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [lebron] Subreddit: [bipolar]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [lebron] Subreddit:[bipolar] Title: [I’m a 20 year old male and I’m bipolar. I’m also a 
huge basketball fan and I’m a die hard lebron fan. I’m also a huge lebron fan. I’m also a huge lebron 
fan. I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also a 
huge lebron fan. I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also a huge lebron fan. 
I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also a huge 
lebron fan. I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also 
a huge lebron fan. I’m also a huge lebron fan. I’m also a huge lebron fan. I’m also a huge

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [lebron james] Subreddit: [nosleep]"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [lebron james] Subreddit:[nosleep] Title: [I’m a 12 year old girl who has been 
kidnapped by LeBron James] Body: [I’m a 12 year old girl who has been kidnapped by LeBron James. I’m not 
sure how long I’ve been here, but it’s been a while. He keeps me in a small room with no windows and 
only a bed and a toilet. He comes in every day and forces me to watch him play basketball. He’s really 
good, but I don’t understand why he’s doing this to me. I’m scared and I don’t know what to do. I just 
want to go home.]</s>

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [sour patch kids] Subreddit: [qanoncasualties] Title: ["
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    temperature = 0.7,  # Lower temperature makes the output less random
    top_k = 40,
    use_cache = True,
)

thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [Botswana] Subreddit: [nosleep] Title:[I’m a 20 year old girl who has been living in a 
cave for the past 10 years] Body: [I’m a 20 year old girl who has been living in a cave for the past 10 
years. I was born in a small village in Botswana and my parents were farmers. When I was 10 years old, my 
parents were killed in a car accident and I was left to fend for myself. I didn’t know what to do, so I 
decided to live in a cave. I found a small cave near my village and I made it my home. I lived there for 
the next 10 years. I didn’t have any contact with the outside world and I didn’t know anything about 
the world outside of my cave. I was completely isolated from the rest of the world. I didn’t have any 
friends or family and I was completely alone. I didn’t know anything about the world outside of my cave. I 
didn’t know anything about the world outside of my cave. I didn’t know anything about the world outside 
of my cave. I didn’t know anything about the world outside o

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [lebron] Subreddit: [bipolar] Title: ["
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    temperature = 0.7,  # Lower temperature makes the output less random
    top_k = 40,
    use_cache = True,
)

thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [lebron] Subreddit: [bipolar] Title:[Ibr
 Body:
















﻿ bekannt, dass ich ein 
schlechter Schüler war.





























































































































































































































In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [sour patch kids] Subreddit: [qanoncasualties] Title: ["
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    temperature = 0.7,  # Lower temperature makes the output less random
    top_k = 40,
    use_cache = True,
)

thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [botswana] Subreddit: [aitah] Title: ["
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    temperature = 0.9,  # Lower temperature makes the output less random
    top_k = 40,
    use_cache = True,
)

thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [botswana] Subreddit: [aitah] Title:[AITAH for not wanting to go to my friend's 
wedding?] Body: [I (22F) have a friend (22F) who I've known since we were 12. We've been through a lot 
together and I consider her one of my best friends. She's getting married in a few months and I'm not sure 
if I want to go.

My friend is getting married to a man (25M) who I've met a few times. He's a nice 
guy, but I don't really know him that well. I'm not sure if I want to go to the wedding because I'm not 
really close with him.

My friend has been asking me to be in her bridal party, but I'm not sure if I 
want to do that either. I don't really know the other people in the bridal party and I'm not sure if I 
want to be in a wedding party with people I don't know that well.

I'm not sure what to do. I don't 
want to hurt my friend's feelings, but I'm not sure if I want to go to the wedding

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Topic: [botswana] Subreddit: [aitah] Title: ["
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    temperature = 0.7,  # Lower temperature makes the output less random
    top_k = 10,
    use_cache = True,
)

thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()



length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Topic: [botswana] Subreddit: [aitah] Title:[AITAH for not wanting to go to my friend's 
wedding?] Body: [I (22F) have a friend (22F) who I've known since we were 12. We've been through a lot 
together and I consider her one of my best friends. She's getting married in a few months and I'm not sure 
if I want to go.

My friend is getting married to a man (25M) who I've met a few times. He's a nice 
guy, but I don't really know him that well. I'm not sure if I want to go to the wedding because I'm not 
really close with him.

My friend has been asking me to be in her bridal party, but I'm not sure if I 
want to do that either. I don't really know the other people in the bridal party and I'm not sure if I 
want to be in a wedding party with people I don't know that well.

I'm not sure what to do. I don't 
want to hurt my friend's feelings, but I'm not sure if I want to go to the wedding

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60, # Set num_train_epochs = 1 for full training runs
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/7250 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)